In [ ]:
#Loading required libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn') #set same style for all plots
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
pd.options.mode.chained_assignment = None
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import randint as sp_randint
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor,BaggingRegressor,AdaBoostRegressor,BaggingRegressor
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
import keras

output = pd.DataFrame(index=None, columns=['model','train_r2_score','test_r2_score'])

Data Import

In [ ]:
data = pd.read_csv('/content/Energy newww.csv')
data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [ ]:
data.columns = ['Relative_Compactness', 'Surface_Area', 'Wall_Area', 'Roof_Area', 'Overall_Height',
                'Orientation', 'Glazing_Area', 'Glazing_Area_Distribution', 'Heating_Load', 'Cooling_Load']

Spliting the data in X and Y

In [ ]:
X = data[['Relative_Compactness', 'Surface_Area', 'Wall_Area', 'Roof_Area', 'Overall_Height',
                'Orientation', 'Glazing_Area', 'Glazing_Area_Distribution']]
Y = data[[ 'Heating_Load', 'Cooling_Load']]
Y1=data[['Heating_Load']]
Y2=data[['Cooling_Load']]
print(X.isnull().sum())

Relative_Compactness         0
Surface_Area                 0
Wall_Area                    0
Roof_Area                    0
Overall_Height               0
Orientation                  0
Glazing_Area                 0
Glazing_Area_Distribution    0
dtype: int64



Dividing each observation in either of the 3 categories (High Efficient, Average Efficient & Low efficient)

In [ ]:
Z = data[['Heating_Load','Cooling_Load']]

# The total load is the sum of the individual heating and cooling load
Z['Overall_load'] = Z['Heating_Load'] + Z['Cooling_Load']
Z['class'] = 1

Z.loc[Z.Overall_load < 42,['class']] = 0
Z.loc[Z.Overall_load > 70 , ['class']] = 2

#Efficiency classes
yclass=Z['class']

X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X, yclass, random_state = 5)

Using MinMax scaler for scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler
MinMax = MinMaxScaler(feature_range= (0,1))
X_train_class = MinMax.fit_transform(X_train_class)
X_test_class = MinMax.transform(X_test_class)

Now dividing the whole dataset into train and test

In [ ]:
#Test-Train split of data for regression models
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 5)

X_train_div, X_test_div, y1_train, y1_test = train_test_split(X, Y1, random_state = 5)

X_train_div, X_test_div, y2_train, y2_test = train_test_split(X, Y2, random_state = 5)


Scaling the data using MinMax Scaler

In [ ]:
# MinMax Scaling done
from sklearn.preprocessing import MinMaxScaler
MinMax = MinMaxScaler(feature_range= (0,1))
X_train = MinMax.fit_transform(X_train)
X_test = MinMax.transform(X_test)

2. Modeling


   Linear Regression


     As the output variables are 2 we will be using Multioutput regressor

In [ ]:
linear = LinearRegression()
multiOutput_linear = MultiOutputRegressor(linear,n_jobs=-1)
multiOutput_linear.fit(X_train,y_train)
train_r2_score=r2_score(y_train,multiOutput_linear.predict(X_train))
test_r2_score=r2_score(y_test,multiOutput_linear.predict(X_test))
output = output.append(pd.Series({'model':'Linear Regressor', 'train_r2_score':train_r2_score,'test_r2_score':test_r2_score}),ignore_index=True )
output

,model,train_r2_score,test_r2_score
0,Linear Regressor,0.902499,0.899426


KNN regressor

In [ ]:
param_grid = {'weights':['distance', 'uniform'], 'n_neighbors':range(3,75)}
grid_search_knn = MultiOutputRegressor(GridSearchCV(KNeighborsRegressor(), param_grid, cv=5,return_train_score=True))
grid_search_knn.fit(X_train, y_train)
train_r2_score=r2_score(y_train,grid_search_knn.predict(X_train))
test_r2_score=r2_score(y_test,grid_search_knn.predict(X_test))
output = output.append(pd.Series({'model':'KNN Regressor', 'train_r2_score':train_r2_score,'test_r2_score':test_r2_score}),ignore_index=True )
output

,model,train_r2_score,test_r2_score
0,Linear Regressor,0.902499,0.899426
1,KNN Regressor,0.935291,0.902266


Random Forest Regressor

In [ ]:
model = RandomForestRegressor( random_state=5, n_estimators= 700, n_jobs = -1)

param_grid = {'max_features':['auto', 'log2'], 'max_depth':[5,10,15,20,30,50,60]}

grid_search_rf = MultiOutputRegressor(GridSearchCV(model, param_grid, cv= 5,return_train_score=True))
grid_search_rf.fit(X_train, y_train)
train_r2_score=r2_score(y_train,grid_search_rf.predict(X_train))
test_r2_score=r2_score(y_test,grid_search_rf.predict(X_test))
output = output.append(pd.Series({'model':'Random Forest Regressor', 'train_r2_score':train_r2_score,'test_r2_score':test_r2_score}),ignore_index=True )
output

,model,train_r2_score,test_r2_score
0,Linear Regressor,0.902499,0.899426
1,KNN Regressor,0.935291,0.902266
2,Random Forest Regressor,0.997694,0.985870


Linear SVM

In [ ]:
param_grid = {"C": [1e0, 1e1, 1e2, 1e3, 1e4],"gamma": np.logspace(-2, 1, 2, 3, 5)}
grid_search_svm = MultiOutputRegressor(GridSearchCV(SVR(kernel='linear'), param_grid, cv=10,return_train_score=True))

grid_search_svm.fit(X_train, y_train)
train_r2_score=r2_score(y_train,grid_search_svm.predict(X_train))
test_r2_score=r2_score(y_test,grid_search_svm.predict(X_test))
output = output.append(pd.Series({'model':'SVM Linear', 'train_r2_score':train_r2_score,'test_r2_score':test_r2_score}),ignore_index=True )
output

,model,train_r2_score,test_r2_score
0,Linear Regressor,0.902499,0.899426
1,KNN Regressor,0.935291,0.902266
2,Random Forest Regressor,0.997694,0.985870
3,SVM Linear,0.897451,0.896480
